In [1]:
import sqlalchemy
import matplotlib.pyplot as plt 
import pandas as pd
pd.option_context('display.max_rows',None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential,load_model
from keras.layers import Dense,LSTM,Bidirectional,GRU
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
add_data_path = r'C:\Users\ahmed\OneDrive\Desktop\CODE_PFE_ETL\prev\rnn\data\convertcsv.csv'
add_d21_path = r'C:\Users\ahmed\OneDrive\Desktop\CODE_PFE_ETL\prev\rnn\data\d2021.csv'
scaler = MinMaxScaler(feature_range=(0,1)) #le scaler est un outil qui permet de normaliser les données en transformant 
#Importation des données de la base MSSQL avec sqlalchemy

In [3]:
def connexion_bd():
    server="localhost"
    database="GIPA"
    driver ="ODBC Driver 17 for SQL Server"
    con = f'mssql://@{server}/{database}?driver={driver}'
    engine = sqlalchemy.create_engine(con,fast_executemany=True)
    con = engine.connect()
    sql = "select Article,DateFacture,CA from GIPA.dbo.faits_resultat"
    df = pd.read_sql(sql,con)  #mise en place des données dans un DataFrame 
    return df

In [4]:
def pretraitement_données(df):
    df['DateFacture'] = pd.to_datetime(df['DateFacture']) #formatage du champ Date
    #groupement de la dataframe en mois
    per = df.DateFacture.dt.to_period("M") 
    g = df.groupby(per) 
    df_clean = g.sum()
    #importation des données de validation
    data_test = pd.read_csv(add_data_path)
    data_test.set_index('Date',inplace=True)
    #jointure des deux df 
    df_finale = pd.concat([data_test,df_clean])
    return df_finale

In [5]:
def transformation_données_2021():
    data21 = pd.read_csv(add_d21_path)
    data21.drop("Date",axis=1,inplace=True)
    array_d21 = np.array(data21)
    return array_d21

In [6]:
def division_données_train_test(df_finale):
    
    dataset = df_finale.values #conversion des valeurs en numpy array   
    train_size = int(len(df_finale)*0.6) #ajustement de la taille d'entrainement du model
    test_size = len(df_finale) - train_size
     #ajustement de la taille de test du model
    #les données entre 0 et 1 ( 0 étant la valeur minime et 1 la valeur maximale) car les LSTM sont plus performants
    #avec des valeurs normalisées
    dataset = scaler.fit_transform(dataset) #application du scaler sur la dataset
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    return dataset ,train, test #division de notre dataset en 2 parties entrainement et test


In [7]:
def création_model(train_gen,test_gen):
    model = Sequential() #utilisation du model de sequence puisque nos données sont en format de series temporelles = sequentiel 
    model.add(Bidirectional(LSTM(164,input_shape=(12,1), activation='relu',return_sequences=True)))
    model.add(LSTM(116,activation='relu'))
    model.add(Dense(1,'linear'))
    model.compile(loss="mse",optimizer=Adam(learning_rate=0.0001),metrics=[RootMeanSquaredError()]) #compilation du model avec la loss function du mean squared error et le adam optimizer
    model.fit_generator(train_gen,validation_data=test_gen,verbose=2,epochs=1000)
    return model

In [8]:
def affichage_des_plots(array_d21,prev,trainY,trainPredict):
    plt.figure(figsize=(20,20))
    ax1 = plt.subplot(3, 1, 1)
    plt.title('Données réelles vs prévision 2021')
    ax1.plot(array_d21,'r-o', label="données réelles")
    ax1.plot(prev,'b-o', label="prévisions") 
    plt.legend()
    ax2 = plt.subplot(3, 1, 2, sharex=ax1)
    plt.title("Données d'entrainement réelles vs prédites")
    ax2.plot(trainY,'-o', label="Données d'entrainement")
    ax2.plot(trainPredict,'.:', label="données prédites d'entrainement") 
    plt.legend()
    plt.show()
     
    result = pd.concat([pd.DataFrame(prev),pd.DataFrame(array_d21)],axis=1)
    l = ['Prévision','Données 2021']
    result.columns = l
    return result